In [1]:
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

sys.path.append('..')

from mim.experiments.ab_glucose import ABGlucose
import mim.extractors.ab_json as ab_json
import mim.models.ab_nn as ab_nn
from mim.config import GLUCOSE_ROOT

In [2]:
specification = {
    "labels": {"target": "label-index+30d-ami+death-30d"},
    "index": {"json_train": "/home/sapfo/andersb/PycharmProjects/Expect/json_data/pontus_glukos/hbg+lund-train.json.gz"},
    "features":{
        "gender",
        "age",
        "bl-Glukos", "bl-TnT", "bl-Krea", "bl-Hb"
    },
}

experiment = ABGlucose.KERAS_LR_BLOOD_BL

In [4]:
dp = experiment.get_data()

{'bl-TnT', 'bl-Hb', 'bl-Glukos', 'bl-Krea', 'age', 'gender'}
['age', 'bl-Glukos', 'bl-Hb', 'bl-Krea', 'bl-TnT']
['gender']
{'bl-TnT', 'bl-Hb', 'bl-Glukos', 'bl-Krea', 'age', 'gender'}
['age', 'bl-Glukos', 'bl-Hb', 'bl-Krea', 'bl-TnT']
['gender']


In [10]:
train = dp.get_set("train")

In [12]:
from mim.extractors.extractor import SingleContainerLinearSplitProvider

In [14]:
sclp = SingleContainerLinearSplitProvider(train, **{"train_frac": 0.6, "val_frac": 0.2, "test_frac": 0.2})

In [17]:
train.lazy_slice(range(500))

In [ ]:
train, val = next(experiment.cross_validation.split(development))

In [ ]:
keras_model = ab_nn.ab_simple_lr(train)

In [ ]:
keras_model.summary()

In [ ]:
len(hold_out)

In [ ]:
compile_args = {
    'optimizer': 'sgd',
    'loss': 'binary_crossentropy',
    'metrics': ['accuracy', tf.keras.metrics.AUC()]
}
keras_model.compile(**compile_args)

In [ ]:
epochs=experiment.epochs
batch_size=experiment.batch_size

x = train['x'].as_dataset
y = train['y'].as_dataset

random_state = experiment.random_state
np.random.seed(random_state)
tf.random.set_seed(random_state)

v = tf.data.Dataset.zip((val['x'].as_dataset, val['y'].as_dataset)).batch(batch_size).prefetch(3)

keras_model.fit(x=tf.data.Dataset.zip((x, y)).batch(batch_size).prefetch(3), 
                validation_data=v,
                epochs=epochs)

In [ ]:
numeric = train['x']['numeric'][:2].copy()